In [30]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

In [31]:
def shape_calculate(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width

In [32]:
class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass

In [33]:
class Conv2dMatrix(ABCConv2d):
    def __call__(self, input_tensor):
        image_size, out_channels, output_height, output_width = shape_calculate(
            input_tensor.shape,
            self.out_channels,
            self.kernel_size,
            self.stride,
            padding=0)

        output_tensor = np.zeros((image_size, out_channels, output_height, output_width))

        for num_image, image in enumerate(input_tensor):

            for num_filter, filter_ in enumerate(self.kernel):

                for i in range(output_height):
                    for j in range(output_width):
                        current_row = self.stride * i
                        current_column = self.stride * j
                        current_slice = image[:, current_row:current_row + self.kernel_size,
                                        current_column:current_column + self.kernel_size]

                        res = float((current_slice * filter_).sum())

                        output_tensor[num_image, num_filter, i, j] = res

        return output_tensor

In [34]:
class MaxPool2D(ABCConv2d):
    def __call__(self, input_tensor):
        image_size, out_channels, output_height, output_width = shape_calculate(
            input_tensor.shape,
            self.out_channels,
            self.kernel_size,
            self.stride,
            padding=0)

        mat_out = np.zeros((image_size, out_channels, output_height, output_width))

        for num_image, image in enumerate(input_tensor):
            for num_chnl in range(image.shape[0]):

                for i in range(output_height):
                    for j in range(output_width):
                        current_row = self.stride * i
                        current_column = self.stride * j
                        current_slice = image[num_chnl, current_row:current_row + self.kernel_size,
                                        current_column:current_column + self.kernel_size]

                        res = float(current_slice.max())
                        mat_out[num_image, num_chnl, i, j] = res
        return mat_out

In [35]:
class LayerNorm:
    def __call__(self, input_tensor, gamma=1, betta=0, eps=1e-3):
        result = np.zeros(input_tensor.shape)
        for b, batch in enumerate(input_tensor):
            for c, image in enumerate(batch):
                Mu = np.mean(image)
                sigma = np.std(image)
                result[b, c, :, :] = ((image - Mu) / (np.sqrt(sigma ** 2 + eps))) * gamma + betta
        return result

In [36]:
class Relu:
    def __call__(self, input_tensor):
        return np.where(input_tensor > 0, input_tensor, 0)

In [37]:
class Model:
    def __init__(self, stride=1):
        kernel = np.random.randint(-10, 10, 5 * 3 * 3).reshape(5, 3, 3)
        in_channels = kernel.shape[1]
        out_channels = kernel.shape[0]
        kernel_size = kernel.shape[2]

        self.conv2d = Conv2dMatrix(in_channels, out_channels, kernel_size, stride)
        self.conv2d.set_kernel(kernel)
        self.LN = LayerNorm()
        self.relu = Relu()
        self.max_pool = MaxPool2D(out_channels, out_channels, 2, 2)

    def model_forward(self, x):
        x = self.conv2d(x)
        x = self.LN(x)
        x = self.relu(x)
        x = self.max_pool(x)
        e_x = np.exp(x - np.max(x, axis=1))
        return e_x / np.sum(e_x, axis=1)

In [38]:
input_img = cv2.imread("kitty.jpg")
input_img = cv2.resize(input_img, (32, 32)).reshape(1, 3, 32, 32)  # BxCxWxH
Net = Model(stride=1)
out = Net.model_forward(input_img)
print(f'Model shape: {out.shape}')
print(f'Out: {out}')

Model shape: (1, 5, 15, 15)
Out: [[[[0.31458357 0.23933641 0.31251125 ... 0.26679685 0.10256506
    0.43685782]
   [0.15202204 0.07597568 0.13753242 ... 0.23922252 0.16341379
    0.60132193]
   [0.08973263 0.2617235  0.11300219 ... 0.27382267 0.16783126
    0.10885295]
   ...
   [0.14183562 0.27543809 0.12588849 ... 0.14048598 0.21817796
    0.22384586]
   [0.17359296 0.17698745 0.22663199 ... 0.18921312 0.22762463
    0.20804418]
   [0.21086758 0.29158118 0.18102773 ... 0.1400576  0.17091706
    0.15023519]]

  [[0.42421457 0.31649551 0.19015968 ... 0.17565942 0.36288548
    0.31510792]
   [0.61704578 0.7421839  0.63967331 ... 0.25065289 0.35719068
    0.17162178]
   [0.59040259 0.3455493  0.58111637 ... 0.0918384  0.19751112
    0.60520249]
   ...
   [0.30375476 0.18173565 0.08160848 ... 0.10999979 0.02803713
    0.1059785 ]
   [0.11436322 0.12747869 0.28062977 ... 0.06709527 0.35963139
    0.17983659]
   [0.40508953 0.33057701 0.06642045 ... 0.10410509 0.04644795
    0.05591694]]

 